In [3]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [14]:
start_date = '2022-10-01'
end_date = '2022-10-31'

In [15]:
Link = f"""


WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(link_lifetime_last_ride_date) 
    link_lifetime_last_ride_date,
    link_lifetime_first_ride_date,
    --MAX(link_lifetime_rides) 
    link_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (link_lifetime_rides > 0 or link_lifetime_rides is not null or link_lifetime_last_ride_date is not null)


    AND link_lifetime_rides BETWEEN 1 AND 4
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN link_lifetime_rides IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR =0'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 AND link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN 'LTR ' || cast(link_lifetime_rides as varchar)
    WHEN link_lifetime_rides BETWEEN 1 AND 4 THEN 'LTR 1-4'
    WHEN (link_lifetime_rides BETWEEN 5 AND 20) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 5-20'
    WHEN (link_lifetime_rides BETWEEN 21 AND 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 21-50'
    WHEN (link_lifetime_rides > 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 50+'
    WHEN link_lifetime_rides >= 5 THEN 'LTR >=5'
    ELSE 'LTR =0'
    END ltr_link,
    CASE 
    WHEN link_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR=0'

    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '0' DAY) THEN '1. 1_7_DAYS'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '1. 8_14_DAYS'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '1. 15_21_DAYS'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '1. 22_30_DAYS'


    WHEN link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '1. LAST_30_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '2. 31_45_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '3. 46_60_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '4. 61_90_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '5. 91_180_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '6. 181_365_DAYS'
    WHEN link_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '7. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_link
FROM base_month_customer
WHERE link_lifetime_rides BETWEEN 1 AND 4 AND link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Link'
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_link,'NEW') ltr_link,
        COALESCE(b.recency_link, 'NEW') recency_link,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_link,
        case 
        when ltr_link in ('LTR 1', 'LTR 2', 'LTR 3', 'LTR 4') then 1
        when ltr_link = 'LTR 1-4' then 2
        when ltr_link = 'LTR 5-20' then 3
        when ltr_link = 'LTR 21-50' then 4
        when ltr_link = 'LTR 50+' then 5
        when ltr_link = 'LTR >=5' then 6
        when ltr_link = 'LTR =0' then 7
        when ltr_link = 'NEW' then 8
        end row_order,
        recency_link,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_link,
        a.recency_link,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_link,
        '' recency_link,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_link,recency_link,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3


"""

In [16]:
df_link = pd.read_sql(Link, conn)
df_link

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR 3,1. 15_21_DAYS,62675,0.46,40502,64.62,36353,58.00,22711,...,26.94,74.36,0.36,34405,0.24,2.04,118141.0,2185442.0,5.405817,3.433832
1,2022-10-01,LTR 2,1. 15_21_DAYS,86483,0.63,51042,59.02,44949,51.97,27013,...,22.90,73.31,0.42,39628,0.28,2.00,167294.0,2516255.0,6.648531,4.221611
2,2022-10-01,LTR 4,1. 15_21_DAYS,49872,0.36,33707,67.59,30642,61.44,19922,...,30.31,75.89,0.32,31296,0.22,2.07,95526.0,1973583.0,4.840232,3.052339
3,2022-10-01,LTR 1,1. 15_21_DAYS,135053,0.98,70980,52.56,59937,44.38,32021,...,16.17,68.19,0.47,41972,0.29,1.92,180567.0,2712856.0,6.655974,4.302082
4,2022-10-01,LTR 3,1. 1_7_DAYS,114468,0.83,93735,81.89,88407,77.23,68921,...,51.60,85.70,1.26,165597,1.16,2.80,556998.0,10081779.0,5.524799,3.363575
5,2022-10-01,LTR 1,1. 1_7_DAYS,174537,1.27,117690,67.43,101992,58.44,62409,...,26.22,73.32,0.98,110166,0.77,2.41,487659.0,6831159.0,7.138745,4.426584
6,2022-10-01,LTR 2,1. 1_7_DAYS,140235,1.02,108968,77.70,100780,71.87,75528,...,45.20,83.93,1.36,161100,1.13,2.54,647850.0,10004262.0,6.475740,4.021415
7,2022-10-01,LTR 4,1. 1_7_DAYS,95552,0.69,80478,84.22,76480,80.04,61348,...,55.86,87.00,1.14,158490,1.11,2.97,495957.0,9517335.0,5.211091,3.129264
8,2022-10-01,LTR 1,1. 22_30_DAYS,173745,1.26,82166,47.29,68852,39.63,35121,...,13.68,67.67,0.51,43211,0.30,1.82,193781.0,2812326.0,6.890417,4.484529
9,2022-10-01,LTR 4,1. 22_30_DAYS,58747,0.43,36766,62.58,33114,56.37,20565,...,26.07,74.47,0.33,30130,0.21,1.97,93648.0,1912682.0,4.896162,3.108131


In [17]:
df_link.to_csv("oct-link-pan--ltr_1-4.csv", index = False)

In [13]:
df_link.to_csv("nov-link-pan--ltr_1-4.csv", index = False)

In [ ]:
df_link['existing_custr_count'].sum()

53898930

In [ ]:
53898930-26949465

26949465

In [ ]:
4461355+431551+257501+633801+319448

6103656

In [ ]:
df_link = pd.read_sql(Link, conn)
df_link

DatabaseError: {'message': 'Encountered too many errors talking to a worker node. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (http://10.16.80.3:8080/v1/task/20221216_183351_01359_t5rsj.10.6/results/1/0 - 242 failures, failure duration 300.63s, total failed request time 307.81s)', 'errorCode': 65540, 'errorName': 'PAGE_TRANSPORT_TIMEOUT', 'errorType': 'INTERNAL_ERROR', 'failureInfo': {'type': 'io.trino.operator.PageTransportTimeoutException', 'message': 'Encountered too many errors talking to a worker node. The node may have crashed or be under too much load. This is probably a transient issue, so please retry your query in a few minutes. (http://10.16.80.3:8080/v1/task/20221216_183351_01359_t5rsj.10.6/results/1/0 - 242 failures, failure duration 300.63s, total failed request time 307.81s)', 'cause': {'type': 'java.io.UncheckedIOException', 'message': 'java.net.SocketTimeoutException: Connect Time

In [ ]:
df_link

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,1642301,6.09,1047277,63.77,925256,56.34,601090,...,29.23,79.86,10.52,1202587,8.00,2.51,5222698.0,73278959.0,7.127145,4.342886
1,2022-09-01,1-4,2. 31_90_DAYS,2127715,7.90,864891,40.65,719026,33.79,366917,...,12.35,71.60,5.76,482735,3.21,1.84,2208773.0,31269240.0,7.063725,4.575539
2,2022-09-01,1-4,3. 91_180_DAYS,2072596,7.69,554664,26.76,452855,21.85,205607,...,6.88,69.39,3.13,247187,1.64,1.73,1249967.0,16264962.0,7.685029,5.056767
3,2022-09-01,1-4,4. 181_365_DAYS,1931183,7.17,305808,15.84,254025,13.15,114891,...,4.07,68.37,1.72,138769,0.92,1.77,731182.0,9241566.0,7.911884,5.269059
4,2022-09-01,1-4,5. 365_ABOVE,5379504,19.96,329247,6.12,278097,5.17,128935,...,1.67,69.52,1.96,168323,1.12,1.88,924223.0,11074306.0,8.345652,5.490771
5,2022-09-01,5-20,1. LAST_30_DAYS,1645938,6.11,1316550,79.99,1237858,75.21,971870,...,51.37,87.00,18.52,2940730,19.55,3.48,10268410.0,170180051.0,6.033851,3.491789
6,2022-09-01,21-50,1. LAST_30_DAYS,721276,2.68,628782,87.18,605129,83.90,526172,...,66.72,91.46,10.54,2487988,16.54,5.17,9193251.0,137760145.0,6.673375,3.695054
7,2022-09-01,50+,1. LAST_30_DAYS,498325,1.85,456247,91.56,445159,89.33,410932,...,77.95,94.53,8.51,3610905,24.01,9.30,15241693.0,193936982.0,7.859096,4.221017
8,2022-09-01,>=5,2. 31_90_DAYS,1810699,6.72,1020455,56.36,905189,49.99,581365,...,25.26,78.68,10.02,1014459,6.74,2.22,3678153.0,62438856.0,5.890808,3.625729
9,2022-09-01,>=5,3. 91_180_DAYS,1192290,4.42,443236,37.18,379312,31.81,211133,...,13.20,74.53,3.45,307853,2.05,1.96,1164385.0,19232267.0,6.054330,3.782276
